In [ ]:
import requests
import io
import os
import time

import numpy as np
import sys
import tensorflow as tf

#librerías para graficar
import matplotlib.pyplot as plt
                 
plt.rcParams['figure.figsize'] = (16, 9)  #ver graficas grandes 
plt.style.use('ggplot')
#guardar las imagenes y tablas en el cuaderno de jupyter
%matplotlib inline 

In [ ]:
print("Version: {}\n Eager mode: {} ".format(tf.__version__,tf.executing_eagerly))
print("GPU is","available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version: 2.7.0
 Eager mode: <function executing_eagerly at 0x7fd145193a70> 
GPU is available


In [ ]:
tf.device('/cpu:0')

#Descarga y procesamiento de datos

In [ ]:
fileUrl = 'https://raw.githubusercontent.com/2HenryCardenas1/DeepLearning/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Datasets/el_rey_arturo_y_los_caballeros_de_la_mesa_redonda.txt'
fileDL= tf.keras.utils.get_file('el_rey_arturo_y_los_caballeros_de_la_mesa_redonda.txt',fileUrl)
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

581632/569235 [==============================] - 0s 0us/step


##Normalizamos el texto

In [ ]:
print("[!] Pasando todo el texto a minuscula")
texto_min = texto.lower()
time.sleep(2)
print("[!] Quitando los saltos de linea del texto")
texto_sin_salto = texto_min.replace('\n', '')
texto_sin_r = texto_sin_salto.replace('\r','\n')
texto_final = texto_sin_r.replace('\n', '')
time.sleep(2)
print("Hecho !!")

[!] Pasando todo el texto a minuscula
[!] Quitando los saltos de linea del texto
Hecho !!


In [ ]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto_final)))
vocab = sorted(set(texto_final))
print('el texto esta compuesto de estos :{} caracteres'. format(len(texto_final)))
print(vocab)
print(texto_final)

el texto tiene longitud de:560854 caracteres
el texto esta compuesto de estos :560854 caracteres
[' ', '!', "'", '(', ')', ',', '-', '.', '0', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'ñ', 'ü', '—']
primero, como uther pendragon envio por el duque de cornualles eigraine su mujer, y de su subita partida otra vezacaecio  en  los  dias  de  uther  pendragon,  cuando  era  este  rey  de  toda inglaterra, y como tal reinaba, que habia un poderoso duque en cornualles que sostenia  guerra  contra  el  mucho  tiempo.  y  el  duque  se  llamaba  duque  de tintagel.  y por  intermedio,  envio  el  rey  uther  por  este  duque,  encargandole que  trajese  consigo  a  su  mujer,  pues  era  tenida  por  hermosa  dama,  y  muy discreta, y se llamaba igraine.cuando  el  duque  y  su  mujer  llegaron  al  rey,  por  intermedio  de  grandes señores  fueron  acordados  ambos.  al 

##Pasamos el texto a numeros


In [ ]:
chars = sorted(list(set(texto_final)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
print(char_to_int,"\n")

n_chars = len(texto_final)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))

[' ', '!', "'", '(', ')', ',', '-', '.', '0', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'ñ', 'ü', '—']
{' ': 0, '!': 1, "'": 2, '(': 3, ')': 4, ',': 5, '-': 6, '.': 7, '0': 8, ':': 9, ';': 10, '?': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'x': 35, 'y': 36, 'z': 37, '¡': 38, '«': 39, '»': 40, '¿': 41, 'ñ': 42, 'ü': 43, '—': 44} 

En total hay 560854 caracteres, y el diccionario tiene un tamaño de 45 caracteres.


###Dividimos el texto en secuencias:
---
Dividimos el texto en estas secuencias, convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente

In [ ]:
# preparar el conjunto de datos de los pares de entrada a salida codificados como enteros
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = texto_final[i:i + seq_length]
	seq_out = texto_final[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Se generaron {} secuencias de texto de un tamaño de {} caracteres". format(n_patterns,seq_length))

Se generaron 560754 secuencias de texto de un tamaño de 100 caracteres


#Entrenamiento

##Preparamos nuestros datos de entrenamiento

---


1.   Primero debemos transformar la lista de secuencias de entrada en la forma [muestras, pasos de tiempo, características] esperada por una red LSTM.
2.   Luego debemos cambiar la escala de los números enteros al rango de 0 a 1 para que los patrones sean más fáciles de aprender mediante la red LSTM que utiliza la función de activación sigmoidea de forma predeterminada.
3.   por ultimo necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 54 caracteres diferentes en el vocabulario (una representación más fácil)


In [ ]:
#transformar la lista X de secuencias de entrada en la forma [muestras, pasos de tiempo, características]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalizar (cambiar la escala de los números enteros al rango de 0 a 1 )
X = X / float(n_vocab)
# convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot.
y = tf.keras.utils.to_categorical(dataY)

##Construcción del modelo RNN

---
definimos nuestro modelo LSTM: 
Aquí definimos una única capa LSTM oculta con 256 unidades de memoria. La red usa deserción con una probabilidad de 20. La capa de salida es una capa densa que usa la función de activación softmax para generar una predicción de probabilidad para cada uno de los 54 caracteres entre 0 y 1.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
#pero agregaremos una segunda capa.
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))

#utilizamos el algoritmo de optimización de ADAM para la velocidad
model.compile(loss='categorical_crossentropy', optimizer='adam')

###Creando chekpoints

---

La red es lenta de entrenar (alrededor de 300 segundos por época) teniendo activa la GPU, ASí que crearemos CHECKPOINTS (puntos de control) para registrar todos los pesos de la red para archivar cada vez que se observe una mejora en la pérdida al final de la época. Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección

In [ ]:
# definimos  los checkpoint
filepath="/content/drive/MyDrive/CheckPoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

###Entrenando

In [ ]:
import os
while True: 
  for i in range(100):
    os.remove('/content/drive/MyDrive/CheckPoints/weights-improvement-{}'.format(i))

In [ ]:
history=model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list, verbose=False)

In [ ]:
to_restore = tf.Variable(callbacks_list)
print(to_restore.numpy())  # Puros ceros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('/content/drive/MyDrive/CheckPoints/'))
new = to_restore.numpy()
print(new)  # Ahora obtenemos el valor restaurado

In [ ]:
status.assert_existing_objects_matched()

##Generando texto con una red LSTM


---
Vamos a cargar el ultimo CHECKPOINT de entrenamiento y con el haremos MAGIA!!!


In [ ]:
filename = "/content/drive/MyDrive/CheckPoints/weights-improvement-1000-0.3400.hdf5" #Aca va el checkpoint con mejor resultado (el ultimo :)
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

###Mapeo inverso (números a letras)
creamos un mapeo inverso que podamos usar para convertir los números enteros nuevamente en caracteres para que podamos entender las predicciones

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

##Predicciones

In [ ]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

##Mejorando la red


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
#pero agregaremos una segunda capa.
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#cambiamos el nombre de archivo de los pesos con puntos de control para que 
#podamos distinguir entre los pesos de esta red 
filepath="/content/drive/MyDrive/CheckPoints/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

###Mejoramos el entrenamiento

---
aumentamos las epoch y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender.

In [ ]:
#los tiempos de entrenamiento aumentaran al doble que en la versión anterior
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

###Haciendo predicciones

In [ ]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"b r i r     l a     v e n t a n a   e n"
tera para 
nejo más  me  padre  ee  las  dos  dl  fuesto  ce  la  inlena.  mo  había  dejado  allí  de  la  toza  de  la  tuelta  despacio  de  su  padre,  después  de  la  cabaña.  después  sodr  me  sonar  dondicson  de  la  terta,  y  le  encontraba  en  la  isla  de  las  cuespos  de  la  quelea  facía  tan delasiada  en  sn  rentir  que  sus  fablza,  desde  había   intentado de  los  tuebajo  de  su  postro  cuando  esteba  mi  pedinin  
aunquemo  de  los  trebvis  do  la  isla  de  las  cuespos  de  la  quelea  facía  tan delasiada  en  sn  rentir  que  sus  fablza,  desde  había   intentado de  los  tuebajo  de  su  postro  cuando  esteba  mi  pedinin  
aunquemo  de  los  trebvis  do  la  isla  de  las  cuespos  de  la  quelea  facía  tan delasiada  en  sn  rentir  que  sus  fablza,  desde  había   intentado de  los  tuebajo  de  su  postro  cuando  esteba  mi  pedinin  
aunquemo  de  los  trebvis  do  la  isla  de  las  cuespo

##Exportar modelo RNN

In [ ]:
!pip install h5py

In [ ]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modeloRNN_cuentosPesos.hdf5")
model.save('modelRNN_cuentos_v_h5.h5')
print("modelo salvado en disco")

###Cargando un modelo

In [ ]:
# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('/content/modelRNN_cuentos_v_h5.h5')

In [ ]:
chars = sorted(list(set("comiendo una manzana")))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))
pattern = dataX[5]
print(pattern)

In [ ]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = new_model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")